In [1]:
import json
import logging
import os
import shutil
from zipfile import ZipFile
import requests
from io import BytesIO
import mimetypes
from time import sleep
from PIL import Image
from typing import List, Tuple
from yaml import dump

log = logging.getLogger(__name__)

In [2]:
class CVAT_API:
    def __init__(self, host, https=False):
        if host.startswith("https://"):
            https = True
        if host.startswith("http://") or host.startswith("https://"):
            host = host.replace("http://", "")
            host = host.replace("https://", "")
        scheme = "https" if https else "http"
        self.base = f"{scheme}://{host}/api/"

    @property
    def tasks(self):
        return self.base + "tasks"

    def tasks_page(self, page_id):
        return self.tasks + "?page={}".format(page_id)

    def tasks_id(self, task_id):
        return self.tasks + "/{}".format(task_id)

    def tasks_id_data(self, task_id):
        return self.tasks_id(task_id) + "/data"

    def tasks_id_dataset(self, task_id, name, fileformat):
        # format fixed to COCO 1.0
        return self.tasks_id(task_id) + "/dataset?filename{}&format={}".format(
            name, fileformat
        )

    def tasks_id_frame_id(self, task_id, frame_id, quality):
        return self.tasks_id(task_id) + "/data?type=frame&number={}&quality={}".format(
            frame_id, quality
        )

    def tasks_id_status(self, task_id):
        return self.tasks_id(task_id) + "/status"

    def tasks_id_annotations_format(self, task_id, fileformat):
        return self.tasks_id(task_id) + "/annotations?format={}".format(fileformat)

    def tasks_id_annotations_filename(self, task_id, name, fileformat):
        return self.tasks_id(task_id) + "/annotations?format={}&filename={}".format(
            fileformat, name
        )

    def get_file_format(self):
        return self.base + "server/annotation/formats"

    @property
    def login(self):
        return self.base + "auth/login"


In [3]:
class CVAT:
    def __init__(self, username: str, password: str, host: str, dump_dir: str):
        self.session = requests.Session()
        self.api = CVAT_API(host=host)
        self.login([username, password])
        self.dump_dir = dump_dir

    def tasks_list(self, use_json_output=True, **kwargs):
        """List all tasks in either basic or JSON format."""
        url = self.api.tasks
        response = self.session.get(url)
        response.raise_for_status()
        output = []
        page = 1
        json_data_list = []
        while True:
            response_json = response.json()
            output += response_json["results"]
            for r in response_json["results"]:
                if use_json_output:
                    json_data_list.append(r)
                else:
                    log.info("{id},{name},{status}".format(**r))
            if not response_json["next"]:
                log.info(json.dumps(json_data_list, indent=4))
                return output
            page += 1
            url = self.api.tasks_page(page)
            response = self.session.get(url)
            response.raise_for_status()

        return output

    def get_file_format(self):
        url = self.api.get_file_format()
        response = self.session.get(url)
        response.raise_for_status()
        return response.json()

    def get_dataset_name(self, task_id):
        """Get dataset name by task id"""
        output = self.tasks_list(use_json_output=True)
        dataset_name = [data["name"]
                        for data in output if data["id"] == task_id]

        return [_dataset_name.replace(" ", "_").lower() for _dataset_name in dataset_name]
        

    def tasks_data(self, task_id, resource_type, resources, **kwargs):
        """Add local, remote, or shared files to an existing task."""
        url = self.api.tasks_id_data(task_id)
        data = {}
        files = None
        if resource_type == "local":
            files = {
                "client_files[{}]".format(i): open(f, "rb")
                for i, f in enumerate(resources)
            }
            print(f"files {files}")
            print(f"resources {resources}")
        elif resource_type == "remote":
            data = {"remote_files[{}]".format(
                i): f for i, f in enumerate(resources)}
        elif resource_type == "share":
            data = {"server_files[{}]".format(
                i): f for i, f in enumerate(resources)}
        data["image_quality"] = 70

        # capture additional kwargs
        for flag in [
            "chunk_size",
            "copy_data",
            "image_quality",
            "sorting_method",
            "start_frame",
            "stop_frame",
            "use_cache",
            "use_zip_chunks",
        ]:
            if kwargs.get(flag) is not None:
                data[flag] = kwargs.get(flag)
        if kwargs.get("frame_step") is not None:
            data["frame_filter"] = f"step={kwargs.get('frame_step')}"

        response = self.session.post(url, data=data, files=files)
        response.raise_for_status()
   
    def tasks_frame(self, task_id, frame_ids, outdir="", quality="original", **kwargs):
        for frame_id in frame_ids:
            url = self.api.tasks_id_frame_id(task_id, frame_id, quality)
            response = self.session.get(url)
            response.raise_for_status()
            im = Image.open(BytesIO(response.content))
            mime_type = im.get_format_mimetype() or "image/jpg"
            im_ext = mimetypes.guess_extension(mime_type)
            if im_ext in (".jpe", ".jpeg", None):
                im_ext = ".jpg"

            outfile = "task_{}_frame_{:06d}{}".format(
                task_id, frame_id, im_ext)
            im.save(os.path.join(outdir, outfile))

    def tasks_dump_annotations(
        self, task_id, fileformat, filename=None, extract=False, remove_zip=False, **kwargs
    ):
        url = self.api.tasks_id(task_id)
        response = self.session.get(url)
        response.raise_for_status()
        response_json = response.json()

        if filename is None:
            filename = 'dataset.zip'

        url = self.api.tasks_id_annotations_filename(
            task_id, filename, fileformat
        )
        while True:
            response = self.session.get(url)
            response.raise_for_status()
            log.info("STATUS {}".format(response.status_code))
            if response.status_code == 201:
                break

        response = self.session.get(url + "&action=download")
        response.raise_for_status()

        dataset_name = os.path.join(self.dump_dir, filename)
        dataset_dir = os.path.join(self.dump_dir, filename.split(".")[0])
        # download annotations (.zip)
        with open(filename, "wb") as fp:
            fp.write(response.content)

        if extract:
            with ZipFile(filename, "r") as zip:
                try:
                    os.makedirs(dataset_dir)
                except:
                    shutil.rmtree(dataset_dir)
                zip.extractall(dataset_dir)

        if remove_zip:
            os.remove(dataset_name)

    def tasks_dump(
        self, task_id, fileformat="COCO 1.0", filename='dataset.zip', extract=False, remove_zip=False, to_fiftyone=False, **kwargs
    ):
        url = self.api.tasks_id(task_id)
        response = self.session.get(url)
        response.raise_for_status()

        url = self.api.tasks_id_dataset(task_id, filename, fileformat)
        if to_fiftyone:
            url = self.api.tasks_id_dataset(task_id, filename, "COCO 1.0")

        while True:
            response = self.session.get(url)
            response.raise_for_status()
            log.info("STATUS {}".format(response.status_code))
            if response.status_code == 201:
                break

        url = "".join(
            (url.split("?")[0], "?action=download&", url.split("?")[-1]))
        response = self.session.get(url)
        response.raise_for_status()

        dataset_name = os.path.join(self.dump_dir, filename)
        dataset_dir = os.path.join(self.dump_dir, filename.split(".")[0])
        # download dataset (.zip)
        with open(dataset_name, "wb") as fp:
            fp.write(response.content)

        if extract:
            with ZipFile(dataset_name, "r") as zip:
                try:
                    os.makedirs(dataset_dir)
                except:
                    shutil.rmtree(dataset_dir)
                zip.extractall(dataset_dir)

        if remove_zip:
            os.remove(dataset_name)

        if to_fiftyone:
            convert_format(dataset_dir, format=fileformat)

    def tasks_export(self, task_id, filename, export_verification_period=3, **kwargs):
        export_url = self.api.tasks_id(task_id) + "/backup"

        while True:
            response = self.session.get(export_url)
            response.raise_for_status()
            log.info("STATUS {}".format(response.status_code))
            if response.status_code == 201:
                break
            sleep(export_verification_period)

        response = self.session.get(export_url + "?action=download")
        response.raise_for_status()

        with open(filename, "wb") as fp:
            fp.write(response.content)
        logger_string = "Task {} has been exported sucessfully. ".format(
            task_id
        ) + "to {}".format(os.path.abspath(filename))
        log.info(logger_string)

    def login(self, credentials):
        url = self.api.login
        auth = {"username": credentials[0], "password": credentials[1]}
        headers = {"Content-Type": "application/json"}
        response = self.session.post(url, json.dumps(auth), headers=headers)
        response.raise_for_status()
        if "csrftoken" in response.cookies:
            self.session.headers["X-CSRFToken"] = response.cookies["csrftoken"]



In [4]:
cvat = CVAT(
    username="admin",
    password="supersecret",
    host="https://gslabel-cvat.xscope.ai",
    # dump_dir="/data1/self_driving/cvat/dump/",
    dump_dir= "./"
)


#hannah / Supersecret1
"""
cvat = CVAT(
    username="hannah",
    password="Supersecret1",
    host="http://da.gsenc.com:8081",
    # dump_dir="/data1/self_driving/cvat/dump/",
    dump_dir= "./"
)
"""

'\ncvat = CVAT(\n    username="hannah",\n    password="Supersecret1",\n    host="http://da.gsenc.com:8081",\n    # dump_dir="/data1/self_driving/cvat/dump/",\n    dump_dir= "./"\n)\n'

In [5]:
import sys
import shutil
import os
from pathlib import Path

import torch
from ultralytics import YOLO

import splitfolders

/work/home/jae/miniforge3/envs/cvat/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class Dataset():
    def __init__(self, images_dirs, dataset_dir="dataset", day=True, night=False):
        self.dataset_dir = dataset_dir
        self.images_dirs = images_dirs
        self.day = day
        self.night = night
        self.missing = []
        self.yaml_path = ""
        
    def get_data(self, labels_dir):
        raw_dataset_dir = self.dataset_dir + "_raw"
        os.makedirs(raw_dataset_dir, exist_ok=True)
        os.makedirs(os.path.join(raw_dataset_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(raw_dataset_dir, "labels"), exist_ok=True)
        for label in os.listdir(labels_dir):
            filename = os.fsdecode(label)[:-4]
            
            image_exists = False
            
            for images_dir in self.images_dirs:
                p = Path(images_dir)
                # print([x for x in p.iterdir() if x.is_dir()])
                for sub_dir in [str(x) for x in p.iterdir() if x.is_dir()]:
                    # print(sub_dir)
                    if sub_dir[len(images_dir) + 1] != "0":
                        continue
                    if self.day:
                        if filename + ".JPG" in os.listdir(os.path.join(sub_dir, "01_주간")):
                            # print("COPIED")
                            shutil.copyfile(os.path.join(images_dir, sub_dir, "01_주간", filename + ".JPG"), os.path.join(raw_dataset_dir, "images", filename + ".JPG"))
                            shutil.copyfile(os.path.join(labels_dir, filename + ".txt"), os.path.join(raw_dataset_dir, "labels", filename + ".txt"))
                            image_exists= True
                    if self.night:
                        if filename + ".JPG" in os.listdir(os.path.join(sub_dir, "02_야간")):
                            shutil.copyfile(os.path.join(images_dir, sub_dir, "02_야간", filename + ".JPG"), os.path.join(raw_dataset_dir, "images", filename + ".JPG"))
                            shutil.copyfile(os.path.join(labels_dir, filename + ".txt"), os.path.join(raw_dataset_dir, "labels", filename + ".txt"))

            if not image_exists:
                self.missing.append(filename)
    
        # splitfolders.ratio(os.path.join(self.dataset_dir, str(task_id)), output=os.path.join(self.dataset_dir, str(task_id)), seed=999, ratio=(0.8, 0.1, 0.1))

    def split(self):
        splitfolders.ratio(self.dataset_dir + "_raw", output=self.dataset_dir, seed=999, ratio=(0.8, 0.1, 0.1))
    
    def make_yaml(self):
        self.yaml_path = os.path.join(self.dataset_dir, "data.yaml")
        with open(os.path.join(self.dataset_dir, "data.yaml"), "w") as f:
            f.write(f"""train: {os.path.join(self.dataset_dir, "train", "images")}
val: {os.path.join(self.dataset_dir, "val", "images")}
test: {os.path.join(self.dataset_dir, "test", "images")}

nc: 2
names: ['TargetArea', 'LineArea']""")

In [7]:
from coco_manager.filter import CocoFilter
from JSON2YOLO.general_json2yolo import convert_coco_json

_task_list = cvat.tasks_list()

cf = CocoFilter()

dataset = Dataset(
    dataset_dir="/work/home/jae/trash_detection/dataset_seg",
    images_dirs=["/data1/trash_detection_combined"]
)

for task in _task_list:
    task_id = task["id"]
    task_name = task["name"]
    segment_size = task["segment_size"]
    status = task["status"]
    jobs = task["jobs"]
    labels = task["labels"]
    print(f"Task ID: {task_id}, Task Name: {task_name}, Status: {status}, Jobs: {jobs}, Labels: {labels}")
    cvat.tasks_dump_annotations(task_id, fileformat="COCO 1.0", filename=f"annotations_seg/{task_id}.zip", extract=True, remove_zip=True)
    # cvat.tasks_dump_annotations(task_id, fileformat="YOLO ZIP 1.0", filename=f"{task_name}.zip", extract=True, remove_zip=True)

    cf.main(f"annotations_seg/{task_id}/annotations/", ["TargetArea", "LineArea"])

    convert_coco_json(json_dir=f"annotations_seg/{task_id}/annotations/", use_segments=True, cls91to80=True, target_dir=f"annotations_seg/{task_id}/annotations/yolo")

    dataset.get_data(f"annotations_seg/{task_id}/annotations/yolo/labels")

Task ID: 36, Task Name: 0628 images-2, Status: annotation, Jobs: {'count': 3, 'completed': 0, 'url': 'https://gslabel-cvat.xscope.ai/api/jobs?task_id=36'}, Labels: {'url': 'https://gslabel-cvat.xscope.ai/api/labels?task_id=36', 'count': 5}


Annotations /work/home/jae/trash_detection/annotations_seg/36/annotations/instances_filtered.json: 100%|█████████████████████████████████████████████████████████████████████| 43/43 [00:00<00:00, 3945.98it/s]


Task ID: 35, Task Name: 0628 images-1, Status: annotation, Jobs: {'count': 4, 'completed': 0, 'url': 'https://gslabel-cvat.xscope.ai/api/jobs?task_id=35'}, Labels: {'url': 'https://gslabel-cvat.xscope.ai/api/labels?task_id=35', 'count': 5}


Annotations /work/home/jae/trash_detection/annotations_seg/35/annotations/instances_filtered.json: 100%|███████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 2108.88it/s]


Task ID: 34, Task Name: 0627 images, Status: annotation, Jobs: {'count': 2, 'completed': 0, 'url': 'https://gslabel-cvat.xscope.ai/api/jobs?task_id=34'}, Labels: {'url': 'https://gslabel-cvat.xscope.ai/api/labels?task_id=34', 'count': 5}


...

Annotations /work/home/jae/trash_detection/annotations_seg/30/annotations/instances_filtered.json: 100%|███████████████████████████████████████████████████████████████████| 242/242 [00:00<00:00, 8369.31it/s]


Task ID: 26, Task Name: 0411 images-1, Status: annotation, Jobs: {'count': 3, 'completed': 0, 'url': 'https://gslabel-cvat.xscope.ai/api/jobs?task_id=26'}, Labels: {'url': 'https://gslabel-cvat.xscope.ai/api/labels?task_id=26', 'count': 5}


Annotations /work/home/jae/trash_detection/annotations_seg/26/annotations/instances_filtered.json: 100%|███████████████████████████████████████████████████████████████████| 211/211 [00:00<00:00, 7427.66it/s]


Task ID: 24, Task Name: 0425 images, Status: validation, Jobs: {'count': 3, 'completed': 1, 'url': 'https://gslabel-cvat.xscope.ai/api/jobs?task_id=24'}, Labels: {'url': 'https://gslabel-cvat.xscope.ai/api/labels?task_id=24', 'count': 5}


Annotations /work/home/jae/trash_detection/annotations_seg/24/annotations/instances_filtered.json: 100%|███████████████████████████████████████████████████████████████████| 199/199 [00:00<00:00, 7120.15it/s]


In [8]:
len(dataset.missing)

255

In [9]:
dataset.split()

Copying files: 1764 files [00:10, 168.05 files/s]


In [10]:
dataset.make_yaml()

In [11]:
class Model():
    def __init__(self):
        torch.cuda.empty_cache()

        self.model = YOLO("models/yolov8n-seg.pt")

    def train(self, data=""):
        self.model.train(
            data=data,              # Path to the dataset configuration file
            epochs=20,              # Number of epochs to train for
            time=None,              # Maximum training time
            patience=10,            # Epochs to wait for no observable improvement for early stopping of training
            batch=16,               # Number of images per batch
            imgsz=640,              # Size of input images as integer
            device=3,               # Device to run on, i.e. cuda device=0 
            project="results",
            optimizer='auto',       # Optimizer to use, choices=[SGD, Adam, Adamax, AdamW, NAdam, RAdam, RMSProp, auto]
            seed=0,                 # Random seed for reproducibility
            close_mosaic=10,        # Disables mosaic data augmentation in the last N epochs
            freeze=None,            # Freezes the first N layers
            lr0=0.01,               # Initial learning rate 
            lrf=0.01,               # Final learning rate (lr0 * lrf)
            momentum=0.937,         # Momentum factor for SGD or beta1 for Adam optimizers
            weight_decay=0.0005,    # l2 regularization term, penalizing large weights to prevent overfitting
            warmup_epochs=3.0,      # Number of epochs for learning rate warmup
            warmup_momentum=0.8,    # Initial momentum for warmup phase
            warmup_bias_lr=0.1,     # Learning rate for bias parameters during warmup phase
            box=7.5,                # Weight of box loss
            cls=0.5,                # Weight of classification loss
            dfl=1.5,                # Weight of distribution focal loss
            dropout=0.0,             # Use dropout regularization
            cache=False
        )

    def save(self):
        self.model.save("models/yolov8-sch.pt")
        # torch.save(self.model, "models/yolov8-sch.pt")

In [12]:
model = Model()

In [13]:
dataset.yaml_path

'/work/home/jae/trash_detection/dataset_seg/data.yaml'

In [14]:
model.train(dataset.yaml_path)

New https://pypi.org/project/ultralytics/8.2.52 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:3 (NVIDIA A100 80GB PCIe, 81051MiB)
engine/trainer: task=segment, mode=train, model=models/yolov8n-seg.pt, data=/work/home/jae/trash_detection/dataset_seg/data.yaml, epochs=20, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=3, workers=8, project=results, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, 

train: Scanning /work/home/jae/trash_detection/dataset_seg/train/labels... 846 images, 0 backgrounds, 0 corrupt: 100%|██████████| 846/846 [00:00<00:00, 970.18it/s]

train: New cache created: /work/home/jae/trash_detection/dataset_seg/train/labels.cache



val: Scanning /work/home/jae/trash_detection/dataset_seg/val/labels... 216 images, 0 backgrounds, 0 corrupt: 100%|██████████| 216/216 [00:00<00:00, 1175.20it/s]


val: New cache created: /work/home/jae/trash_detection/dataset_seg/val/labels.cache
Plotting labels to results/train11/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to results/train11
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      2.75G     0.3212     0.2065      0.239     0.9658         28        640: 100%|██████████| 53/53 [00:28<00:00,  1.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.74it/s]

                   all        216        431      0.996          1      0.993      0.944      0.996          1      0.993      0.961



20 epochs completed in 0.170 hours.
Optimizer stripped from results/train11/weights/last.pt, 6.8MB
Optimizer stripped from results/train11/weights/best.pt, 6.8MB

Validating results/train11/weights/best.pt...
Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:3 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8n-seg summary (fused): 199 layers, 3258454 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:03<00:00,  1.82it/s]


                   all        216        431      0.994          1      0.994      0.947      0.994          1      0.994      0.961
            TargetArea        216        215      0.992          1      0.992      0.935      0.992          1      0.992      0.943
              LineArea        216        216      0.995      0.999      0.995      0.958      0.995      0.999      0.995      0.979
Speed: 1.4ms preprocess, 0.8ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to results/train11


In [20]:
import cv2

img = cv2.imread("/data1/trash_detection_Pyeongtaek_Ilsan/0628 평택/02_야간/G-N-0628-00076.JPG")

model.model.predict(img)

TypeError: eval() takes 1 positional argument but 2 were given

In [123]:
for result in results:
    boxes = result.boxes  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    probs = result.probs  # Class probabilities for classification outputs

In [15]:
model.model.val(data=dataset.yaml_path)

Ultralytics YOLOv8.1.34 🚀 Python-3.9.19 torch-2.0.1+cu117 CUDA:3 (NVIDIA A100 80GB PCIe, 81051MiB)
YOLOv8n-seg summary (fused): 199 layers, 3258454 parameters, 0 gradients, 12.0 GFLOPs


val: Scanning /work/home/jae/trash_detection/dataset_seg/test/labels... 225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<00:00, 1063.77it/s]

val: New cache created: /work/home/jae/trash_detection/dataset_seg/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:10<00:00,  1.39it/s]


                   all        225        450      0.997      0.996      0.994      0.946      0.997      0.996      0.994      0.967
            TargetArea        225        226          1      0.996      0.995      0.941          1      0.996      0.995       0.96
              LineArea        225        224      0.994      0.996      0.992      0.951      0.994      0.996      0.992      0.975
Speed: 0.8ms preprocess, 2.8ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to results/train112


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f089c0b4c40>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04104